In [1]:
# import des librairies 
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import pickle

# clean text data
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

# model 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-08-23 08:37:36.127840: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-23 08:37:36.491989: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-23 08:37:36.493789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 08:37:38.056010: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# import des données
jobs_df = pd.read_csv("../data/silver_data.csv")
jobs_salary = pd.read_csv("../data/job_salary.csv")

display(jobs_df.head(5))
print(jobs_df.shape)
display(jobs_salary.head())
print(jobs_salary.shape)

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"analyser les résultats d'un projet, décliner l...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,contrôler et faire appliquer le respect de dis...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13
3,Ingénieur Data H/F,"Vos missions\nAu sein de projets Agile Scrum, ...",66 - PERPIGNAN,Actualisé le 19 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,"Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,100 à 199 salariés,"Azura, est un groupe familial, en forte croiss...","tableau, sas",66,PERPIGNAN,2023/07/19
4,Data Engineer pour la Qualité Totale H/F (H/F),Vous êtes Rattaché(e) au Responsable de l'équi...,92 - ST CLOUD,Actualisé le 19 juillet 2023,35H Travail en journée,38000.0 - 70000.0,Contrat à durée indéterminée,5 ans,NaN,"application web, déterminer des mesures correc...",Cadre,Activités des sièges sociaux,DASSAULT AVIATION,Non employeur (pas de salarié au cours de l'an...,Dassault Aviation,NaN,92,ST CLOUD,2023/07/19


(2613, 19)


,Job,salary($),salary_type,Base Salary,Bonus,Total Pay,Profit Sharing,Commission,Hourly Rate
0,ABAP Developer,90000.0,year,$68k - $120k,$2k - $20k,$67k - $132k,NaN,NaN,NaN
1,ASIC Design Engineer,108330.0,year,$83k - $148k,$1k - $15k,$85k - $164k,$2k - $15k,NaN,NaN
2,ASIC Engineer,116546.0,year,$59k - $165k,$503 - $22k,$59k - $179k,NaN,NaN,NaN
3,ASP.NET Developer,71152.0,year,$50k - $94k,$993 - $10k,$47k - $96k,$2k - $5k,NaN,NaN
4,Actuarial Associate,82113.0,year,$65k - $101k,$2k - $13k,$67k - $109k,$2k - $7k,NaN,NaN


(661, 9)


### Clean des données

In [3]:
# préparer un df avec uniquement les colonnes à utiliser pour le modèle
df = jobs_df[["jobs", "description", "ID_dep", "ville", "date", "experience", "skills", "tools", "industry", "company", "company_description"]]
df.head()

,jobs,description,ID_dep,ville,date,experience,skills,tools,industry,company,company_description
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75,PARIS 08,2023/07/12,5 ans,"concevoir et gérer un projet, concevoir un log...",NaN,Conseil pour les affaires et autres conseils d...,QUANTMETRY,Pure player en Data et Intelligence Artificiel...
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75,PARIS 08,2023/07/12,5 ans,"analyser les résultats d'un projet, décliner l...",NaN,Conseil pour les affaires et autres conseils d...,QUANTMETRY,Pure player en Data et Intelligence Artificiel...
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92,ASNIERES SUR SEINE,2023/07/13,3 ans,contrôler et faire appliquer le respect de dis...,tableau,Activités des sièges sociaux,NEXITY,Nexity est aujourd hui leader sur les différen...
3,Ingénieur Data H/F,"Vos missions\nAu sein de projets Agile Scrum, ...",66,PERPIGNAN,2023/07/19,5 ans,"concevoir et gérer un projet, concevoir un log...","tableau, sas","Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,"Azura, est un groupe familial, en forte croiss..."
4,Data Engineer pour la Qualité Totale H/F (H/F),Vous êtes Rattaché(e) au Responsable de l'équi...,92,ST CLOUD,2023/07/19,5 ans,"application web, déterminer des mesures correc...",NaN,Activités des sièges sociaux,DASSAULT AVIATION,Dassault Aviation


In [4]:
df["jobs"] = df["jobs"].str.replace(" h/f", "")
df["jobs"] = df["jobs"].str.replace("\s*\([hH]/[fF]\)\s*", " ").str.strip()
df["jobs"] = df["jobs"].str.replace(" (H/F)", "")
df["jobs"] = df["jobs"].str.replace(" H/F", "")
df["jobs"] = df["jobs"].str.replace(" F/H", "")
df["jobs"] = df["jobs"].str.replace(" (F/H)", "")
df["jobs"] = df["jobs"].str.replace(" \(\)", "")
df["jobs"] = df["jobs"].str.replace(" CDI", "")
df["jobs"] = df["jobs"].str.replace(" CDD", "")

/tmp/ipykernel_2279/841066638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["jobs"] = df["jobs"].str.replace(" h/f", "")
/tmp/ipykernel_2279/841066638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["jobs"] = df["jobs"].str.replace("\s*\([hH]/[fF]\)\s*", " ").str.strip()
/tmp/ipykernel_2279/841066638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [5]:
df.head()

,jobs,description,ID_dep,ville,date,experience,skills,tools,industry,company,company_description
0,Data Engineer sénior,"En tant que Data Engineer chez Quantmetry, vou...",75,PARIS 08,2023/07/12,5 ans,"concevoir et gérer un projet, concevoir un log...",NaN,Conseil pour les affaires et autres conseils d...,QUANTMETRY,Pure player en Data et Intelligence Artificiel...
1,Data Consultant Stratégie Sénior,Nous recrutons des personnes avec une appétenc...,75,PARIS 08,2023/07/12,5 ans,"analyser les résultats d'un projet, décliner l...",NaN,Conseil pour les affaires et autres conseils d...,QUANTMETRY,Pure player en Data et Intelligence Artificiel...
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92,ASNIERES SUR SEINE,2023/07/13,3 ans,contrôler et faire appliquer le respect de dis...,tableau,Activités des sièges sociaux,NEXITY,Nexity est aujourd hui leader sur les différen...
3,Ingénieur Data,"Vos missions\nAu sein de projets Agile Scrum, ...",66,PERPIGNAN,2023/07/19,5 ans,"concevoir et gérer un projet, concevoir un log...","tableau, sas","Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,"Azura, est un groupe familial, en forte croiss..."
4,Data Engineer pour la Qualité Totale,Vous êtes Rattaché(e) au Responsable de l'équi...,92,ST CLOUD,2023/07/19,5 ans,"application web, déterminer des mesures correc...",NaN,Activités des sièges sociaux,DASSAULT AVIATION,Dassault Aviation


In [7]:
# Prétraitement des données
def preprocess_text(text):
    # Suppression des caractères spéciaux et de la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Conversion en minuscules
    text = text.lower()
    
    # Suppression des mots vides
    stop_words = set(stopwords.words('french'))  
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Rejoindre les tokens prétraités en une seule chaîne
    processed_text = ' '.join(tokens)
    
    return processed_text

In [7]:
df["description"][0]

"En tant que Data Engineer chez Quantmetry, vous serez amené à travailler sur des problématiques de :  \n\nIngestion de données (collecte push/pull stream, batch), règles de qualité, traçabilité, conformité et confidentialié. \n\nMise en œuvre de pipeline de données permettant de gérer les normalisations, enrichissements, mise en qualité, calculs de KPI   \n\nMise en œuvre d'outils de stockage appropriés (base de données SQL, No-SQL, stockage fichier ou objets) en fonction de la performance ou du coût \n\nModélisation des données, dès la collecte jusqu'à l'exposition \n\nTraiter, modéliser, stocker, interroger et visualiser les données pour les fournir à des utilisateurs métiers \n\nConception et développement d'APIs pour rendre disponible les données  \n\nMettre en place et configurer les services de monitoring techniques, applicatifs et flux \n\nMettre en place et utiliser les outils d'orchestrations et de CI/CD  \n\nParticiper à l'industrialisation et l'optimisation d'algorithmes de

In [8]:
df["description"] = df["description"].apply(preprocess_text)
df["company_description"] = df["company_description"].astype(str)
df["company_description"] = df["company_description"].apply(preprocess_text)
df["experience"] = df["experience"].apply(preprocess_text)
df["skills"] = df["skills"].astype(str)
df["skills"] = df["skills"].apply(preprocess_text)

/tmp/ipykernel_1722/1014681133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["description"] = df["description"].apply(preprocess_text)
/tmp/ipykernel_1722/1014681133.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["company_description"] = df["company_description"].astype(str)
/tmp/ipykernel_1722/1014681133.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [9]:
df["skills"][0]

'concevoir gérer projet concevoir logiciel système dinformations application évaluer résultat action'

In [10]:
df.fillna("aucune valeur", inplace=True)

/tmp/ipykernel_1722/2316103424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna("aucune valeur", inplace=True)


In [11]:
df['company_description'] = df['company_description'].replace('nan', 'aucune description')

/tmp/ipykernel_1722/3294830435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['company_description'] = df['company_description'].replace('nan', 'aucune description')


In [12]:
df.head()

,jobs,description,ID_dep,ville,date,experience,skills,tools,industry,company,company_description
0,Data Engineer sénior,tant data engineer chez quantmetry amené trava...,75,PARIS 08,2023/07/12,5 an,concevoir gérer projet concevoir logiciel syst...,aucune valeur,Conseil pour les affaires et autres conseils d...,QUANTMETRY,pure player data intelligence artificielle qua...
1,Data Consultant Stratégie Sénior,recrutons personnes appétence forte data lia c...,75,PARIS 08,2023/07/12,5 an,analyser résultats dun projet décliner concept...,aucune valeur,Conseil pour les affaires et autres conseils d...,QUANTMETRY,pure player data intelligence artificielle qua...
2,Chef de projets Performance Durable/Energie/Da...,intégrée sein direction performance durable ne...,92,ASNIERES SUR SEINE,2023/07/13,3 an,contrôler faire appliquer respect disposition ...,tableau,Activités des sièges sociaux,NEXITY,nexity aujourd hui leader différents métiers i...
3,Ingénieur Data,mission sein projets agile scrum mission défin...,66,PERPIGNAN,2023/07/19,5 an,concevoir gérer projet concevoir logiciel syst...,"tableau, sas","Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,azura groupe familial forte croissance spécial...
4,Data Engineer pour la Qualité Totale,rattachée responsable léquipe méthodes support...,92,ST CLOUD,2023/07/19,5 an,application web déterminer mesures corrective,aucune valeur,Activités des sièges sociaux,DASSAULT AVIATION,dassault aviation


### Model

In [13]:
tools = df["tools"].values
job_description = df["description"].values
ville = df["ville"].values
industry = df["industry"].values

job_title = df["jobs"].values
# Enlever les espaces vides à la fin
job_title = np.array([title.rstrip() for title in job_title])

In [14]:
def create_sequences(description, tools, industry, job_title, num_features_input, num_features_target, seq_length, num_seqs):
    """
    Crée des séquences pour le modèle encoder-décoder.
    
    Parameters:
    - description, tools, industry, job_title : Les données sources
    - num_features_input : Le nombre de features pour l'input
    - num_features_target : Le nombre de features pour le target
    - seq_length : La longueur de chaque séquence
    - num_seqs : Le nombre de séquences à créer
    
    Returns:
    - input_sequences, target_sequences : Les séquences d'entrée et de sortie pour le modèle
    """

    # Création des séquences d'entrée
    input_sequences = []
    for i in range(num_seqs):
        combined_seq = []
        combined_seq.extend(description[i*num_features_input:(i+1)*num_features_input])
        combined_seq.extend(tools[i*num_features_input:(i+1)*num_features_input])
        combined_seq.extend(industry[i*num_features_input:(i+1)*num_features_input])
        
        input_sequences.append(combined_seq[:seq_length])  # Garantir que la séquence ne dépasse pas la longueur définie
    
    # Création des séquences de sortie
    target_sequences = [job_title[i*num_features_target:(i+1)*num_features_target] for i in range(num_seqs)]
    
    return input_sequences, target_sequences

In [37]:
# 1. Créez les séquences avec la fonction fournie
input_sequence, target_sequence = create_sequences(job_description, tools, industry, job_title, 1, 1, 3, 2613)  # Ajustez les valeurs selon vos besoins

In [38]:
input_sequence[0]

['tant data engineer chez quantmetry amené travailler problématiques ingestion données collecte pushpull stream batch règles qualité traçabilité conformité confidentialié mise œuvre pipeline données permettant gérer normalisation enrichissements mise qualité calculs kpi mise œuvre doutils stockage appropriés base données sql nosql stockage fichier objets fonction performance coût modélisation données dès collecte jusquà lexposition traiter modéliser stocker interroger visualiser données fournir utilisateurs métiers conception développement dapis rendre disponible données mettre place configurer service monitoring technique applicatifs flux mettre place utiliser outils dorchestrations cicd participer lindustrialisation loptimisation dalgorithmes machine learning mettre action gestion cycle vie modèlesia développements réalisés infrastructure onpremise environnements cloud afin dassurer haut niveau dexigence environnements nécessitent prendre compte sécurité authentification autorisation

In [39]:
target_sequence[0]

array(['Data Engineer sénior'], dtype='<U150')

In [40]:
# 2. Tokenization
tokenizer = Tokenizer(filters='')  # Pour éviter le filtrage de certains caractères
tokenizer.fit_on_texts(['<start>', '<end>'])  # Ajouter les tokens spéciaux en premier

input_texts = [" ".join(map(str, seq)) for seq in input_sequence]
target_texts = ["<start> " + " ".join(map(str, seq)) + " <end>" for seq in target_sequence]

tokenizer.fit_on_texts(input_texts + target_texts)

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

max_sequence_length = max([len(seq) for seq in input_sequences + target_sequences])

input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_sequence_length, padding='post')

vocab_size = len(tokenizer.word_index) + 1

In [41]:
# Define the input layers
encoder_inputs = Input(shape=(max_sequence_length,))
decoder_inputs = Input(shape=(max_sequence_length-1,))

# Define model architecture
latent_dim = 50 # Dimensionality of the latent space
embedding = Embedding(vocab_size, latent_dim)

encoder_embedding = embedding(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_embedding = embedding(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Create the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

assert len(input_sequence) == len(target_sequence), "Mismatched sequence lengths!"

# Jusqu'ici, tout est identique. Maintenant, divisons les données:
input_train, input_val, target_train, target_val = train_test_split(input_sequences, target_sequences, test_size=0.2, random_state=42)

# Préparez les séquences d'entraînement et de validation :
target_train_input = target_train[:, :-1]
target_train_output = target_train[:, 1:]
target_train_output = np.expand_dims(target_train_output, -1)

target_val_input = target_val[:, :-1]
target_val_output = target_val[:, 1:]
target_val_output = np.expand_dims(target_val_output, -1)

# Train the model
baseline = model.fit([input_train, target_train_input], 
                     target_train_output,
                     validation_data=([input_val, target_val_input], target_val_output),
                     epochs=100, 
                     batch_size=8)

Epoch 1/100
 28/262 [==>...........................] - ETA: 4:24 - loss: 8.9621

KeyboardInterrupt: 

In [ ]:
baseline_df = pd.DataFrame(baseline.history)
baseline_df.loc[:, ['loss', 'val_loss']].plot(title="MSE")
print("MSE: {:0.4f}".format(baseline_df['val_loss'].min()))

In [ ]:
model.save('../API/model.h5')

# To save tokenizer
tokenizer_json = tokenizer.to_json()
with open('../API/tokenizer.json', 'w') as file:
    file.write(tokenizer_json)

In [90]:
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 472)]                0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 473)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     multiple                     1013200   ['input_5[0][0]',             
                                                                     'input_6[0][0]']             
                                                                                                  
 lstm_4 (LSTM)               [(None, 50),                 20200     ['embedding_2[0][0]']   

### Generate text

In [22]:
from tensorflow.keras.models import load_model

model_path = '../API/model.h5'
model = load_model(model_path)

from tensorflow.keras.preprocessing.text import tokenizer_from_json

# Charger le tokenizer depuis le fichier JSON
tokenizer_path = '../API/tokenizer.json'
with open(tokenizer_path, 'r') as file:
    tokenizer_json_content = file.read()
    tokenizer = tokenizer_from_json(tokenizer_json_content)

In [28]:
# Validate presence of <start> token
assert '<start>' in tokenizer.word_index, "<start> token not found in tokenizer"

# Encoder model
encoder_inputs = model.get_layer('input_5').output
encoder_lstm = model.get_layer('lstm_4')
encoder_states = encoder_lstm.output[1:]
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model
latent_dim = model.get_layer('lstm_4').units

decoder_inputs = model.get_layer('input_6').output
decoder_embedding_layer = model.get_layer('embedding_2')
decoder_embedding_ = decoder_embedding_layer(decoder_inputs)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_layer = model.get_layer('lstm_5')
decoder_outputs_, state_h_, state_c_ = decoder_lstm_layer(decoder_embedding_, initial_state=decoder_states_inputs)

decoder_dense_layer = model.get_layer('dense_2')
decoder_outputs_ = decoder_dense_layer(decoder_outputs_)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs_] + [state_h_, state_c_])


def generate_text(input_sequence):
    states_value = encoder_model.predict(input_sequence)
    target_sequence = np.zeros((1, 1))
    target_sequence[0, 0] = tokenizer.word_index['<start>']
    
    stop_condition = False
    generated_text = []
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_sequence] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, '')
        
        if sampled_word == '<end>':
            stop_condition = True
        else:
            generated_text.append(sampled_word)

        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index
        states_value = [h, c]
    
    return ' '.join(generated_text)

In [33]:
# Get user inputs
feature_1 = input("Enter tools: ")
feature_2 = input("Enter a description of you're dream job: ")

Enter tools:  la végétation
Enter a description of you're dream job:  le vert


In [34]:
# Combine features into input sequence
input_sequence = f"{feature_1} {feature_2}"

# Convert string features to tokenized sequences
input_sequence_sequence = tokenizer.texts_to_sequences([input_sequence])
max_sequence_length = 473

# Pad the sequence
input_sequence_padded = pad_sequences(np.array(input_sequence_sequence), maxlen=max_sequence_length, padding='post')

# Generate text
generated_text = generate_text(input_sequence_padded)
print(generated_text)

1/1 [==============================] - 0s 20ms/step
data data
